In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xlrd
import pandas as pd
from scipy.stats import weibull_min
import scipy

%matplotlib inline

In [1]:
class RepairStockModel(object):
    
    def __init__(self, t=None, i=None, o=None, s=None, lt=None, s_c=None, o_c=None, name='DSM', sf=None,
                r_r = None, r_lt = None, r_sf = None, r_s_c = None, r_s=None, r_i = None, r_i_sf = None):
        
        """Init function. Assign the input data to the instance of the object."""
        
        self.t = t 
        self.i = i
        self.s = s  
        self.s_c = s_c
        self.o = o  
        self.o_c = o_c  
        self.lt = lt
        self.sf  = sf
        self.name = name
        
        self.r_r = r_r
        self.r_lt = r_lt       
        self.r_sf = r_sf
        self.r_s_c = r_s_c
        self.r_s = r_s
        self.r_i = r_i
        self.r_i_sf = r_i_sf
        
        
        if lt is not None:
            for ThisKey in lt.keys():
                # If we have the same scalar lifetime, stdDev, etc., for all cohorts,
                # replicate this value to full length of the time vector
                if ThisKey != 'Type':
                    if np.array(lt[ThisKey]).shape[0] == 1:
                        lt[ThisKey] = np.tile(lt[ThisKey], len(t))
                        
        if r_lt is not None:
            for ThisKey in r_lt.keys():
                # If we have the same scalar lifetime, stdDev, etc., for all cohorts,
                # replicate this value to full length of the time vector
                if ThisKey != 'Type':
                    if np.array(r_lt[ThisKey]).shape[0] == 1:
                        r_lt[ThisKey] = np.tile(r_lt[ThisKey], len(t))

        
        
    def calculate_survival_function(self):
        """Determine survival function of a cohort per year taken from lifetime distribution."""
        
        self.sf = np.zeros((len(self.t), len(self.t)))
        
        for m in range(0, len(self.t)):  # cohort index
            if self.lt['Shape'][m] != 0:  # For products with lifetime of 0, sf == 0
                self.sf[m::,m] = scipy.stats.weibull_min.sf(np.arange(0,len(self.t)-m), c=self.lt['Shape'][m], loc = 0, scale=self.lt['Scale'][m])
        
        return self.sf
    
    def calculate_repaired_survival_function(self):
        """Determine survival function of the repaired cohort taken from the secondary lifetime distribution."""
        
        self.r_sf = np.zeros((len(self.t), len(self.t)))
        
        for m in range(0, len(self.t)):  # cohort index
            if self.r_lt['Shape'][m] != 0:  # For products with lifetime of 0, sf == 0
                self.r_sf[m::,m] = scipy.stats.weibull_min.sf(np.arange(0,len(self.t)-m), c=self.r_lt['Shape'][m], loc = 0, scale=self.r_lt['Scale'][m])

        return self.r_sf
    
    
    def calculate_repaired_inflow_decay(self, m, r_i):
        """Calculate the repaired inflow decay of devices which have underwent repair"""
        
        for n in range(0, len(self.t)):
            
            start_fill = m*len(self.t)
            
            self.r_i_sf[m:,start_fill+n] = self.r_i[m,n] * self.r_sf[m:,m]
            
        return self.r_i_sf
        
    
    
    def calculate_repaired_stock_by_cohort(self,m, r_i_sf):
        """Calculate the repaired cohort composition of devices which have underwent repair"""
        
        for n in range (0, len(self.t)):
            
            fillStart = (n)* len(self.t)
            fillEnd = (n+1) * len(self.t)
            
            self.r_s_c[m,n] = self.r_i_sf[m, fillStart:fillEnd].sum()
        
        return self.r_s_c
    
    
    def calculate_repaired_stock(self, m, r_i_sf):
        """Calculate the repaired stock"""
        
        self.r_s[m] = r_i_sf[m,:].sum()
        
        return self.r_s
        

        
    def compute_stock_driven_model(self):
        """ With given total stock and lifetime distribution, 
                the method builds the stock by cohort and the inflow.
        """
        if self.s is not None:
            if self.lt is not None:
                
                self.i = np.zeros(len(self.t))
                self.s_c = np.zeros((len(self.t), len(self.t)))
                self.o_c = np.zeros((len(self.t), len(self.t)))
                
                self.r_i = np.zeros((len(self.t), len(self.t)))
                self.r_i_sf = np.zeros(((len(self.t),(len(self.t)*len(self.t)))))
                self.r_s = np.zeros(len(self.t))
                self.r_s_c = np.zeros((len(self.t), len(self.t)))
                
                
                # construct the sf of a product of cohort tc remaining in the stock in year t
                self.calculate_survival_function()
                self.calculate_repaired_survival_function()
                
                # First year:
                if self.sf[0, 0] != 0: # Else, inflow is 0.
                    self.i[0] = self.s[0] / self.sf[0, 0]
                
                self.s_c[:, 0] = self.i[0] * self.sf[:, 0] # Future decay of age-cohort of year 0.
                self.o_c[0, 0] = self.i[0] - self.s_c[0, 0]
                
                self.r_i[0, 0] = self.o_c[0, 0] * self.r_r[0] #Repaired devices enter stock at beginning of next year
                 
                self.r_i_sf = self.calculate_repaired_inflow_decay(m=0, r_i = self.r_i)
                self.r_s_c = self.calculate_repaired_stock_by_cohort(m=0, r_i_sf = self.r_i_sf)

                
                # all other years:
                for m in range(1, len(self.t)):  # for all years m, starting in second year   
                    
                    #Compute outflow from previous age-cohorts up to m-1                    
                    self.o_c[m,:] = self.s_c[m-1,:] - self.s_c[m,:]

                    #calculate how much outflow can get repaired
                    self.r_i[m,:] = self.o_c[m,:] * self.r_r[m]
                    
                    #calculate the decay of this repaired outflow
                    self.r_i_sf = self.calculate_repaired_inflow_decay(m=m, r_i = self.r_i)
                    
                    #calculate the stock by cohort of the repaired devices
                    self.r_s_c = self.calculate_repaired_stock_by_cohort(m=m, r_i_sf = self.r_i_sf)
                    
                    self.r_s[m] = self.r_i_sf[m,:].sum()

                    
                    #Consistency checks are needed to ensure the model is sensible
                    #The first check is to determine if the lifetime parameters are accurate
                    #This test is called a negative inflow test, which is needed if the decay is too slow i.e. the modelled stock is larger than given stock
                    
                    InflowTest = self.s[m] - self.s_c[m,:].sum() - self.r_s_c[m-1,:].sum()   #-1 as the inflow and therefore the stock of year m is actually the i and s for next year

                    if InflowTest >= 0: #input parameters (lifetime and repair rate) are sensible as a positive inflow is needed

                        #as an inflow is needed, the devices can either be new and repaired or just repaired
                        #the first instance occurs if the InflowTest value is larger than the the repaired inflow
                        #the second instance occurs if InflowTest is smaller than repaired inflow
                        
                        if InflowTest > self.r_i[m,:].sum(): #first scenario, new and repaired
                            
                            self.i[m] = (self.s[m] - self.s_c[m,:].sum() - self.r_s_c[m-1,:].sum())
                            self.s_c[m::, m] = self.i[m] * self.sf[m::, m]
                            
                            
                            self.r_i_sf = self.calculate_repaired_inflow_decay(m=m, r_i = self.r_i)
                            self.r_s_c = self.calculate_repaired_stock_by_cohort(m=m, r_i_sf = self.r_i_sf)

                        else: #second scenario, repair rate is too much, all the inflow should come from repaired devices, but repaired total must decrease in propotion to what is needed
                           
                            self.i[m] = 0
                            self.s_c[m::, m] = self.i[m] * self.sf[m::, m]
                            
                            Repair_Reduction = InflowTest/(self.r_i[m-1,:].sum()) #how much should repaired inflow be reduced by across all cohorts
                            
                            self.r_i[m-1,:] = self.r_i[m-1,:]*Repair_Reduction #decreasing repaired inflow across all cohorts
                            
                            self.r_i_sf = self.calculate_repaired_inflow_decay(m=m, r_i = self.r_i)
                            self.r_s_c = self.calculate_repaired_stock_by_cohort(m=m, r_i_sf = self.r_i_sf)

                    else: #lifetime paramters are not sensible, a negative inflow is needed
                          #therefore the total stock needs to decrease by increasing the outflow
                          #the increase in outflow will NOT cause an increase in repair inflow
                        
                        Delta = -1 * InflowTest # Delta > 0!
                        self.i[m] = 0 # Set inflow to 0 and distribute mass balance gap onto remaining cohorts:

                        if self.s_c[m,:].sum() != 0:
                            Delta_percent = Delta / (self.s_c[m,:].sum() + self.r_s_c[m-1,:].sum())
                            # Distribute gap equally across all cohorts (each cohort is adjusted by the same %, based on surplus with regards to the prescribed stock)
                            # Delta_percent is a % value <= 100%

                        else:                            

                            Delta_percent = 0 # stock in this year is already zero, method does not work in this case.
                            # correct for outflow and stock in current and future years
                            # adjust the entire stock AFTER year m as well, stock is lowered in year m, so future cohort survival also needs to decrease.

                        self.o_c[m, :] = self.o_c[m, :] + (self.s_c[m, :] * Delta_percent)  # increase outflow according to the lost fraction of the stock, based on Delta_c
                        self.s_c[m::,0:m] = self.s_c[m::,0:m] * (1-Delta_percent) # shrink future description of stock from previous age-cohorts by factor Delta_percent in current AND future years.
                        self.r_s_c[m-1::,0:m] = self.r_s_c[m-1::,0:m] * (1-Delta_percent) #decrease future repaired stock by factor Delta_percent                

                
                
                #Shift the repaired values down by a row of zeros and remove anything > len(self.t)
                self.r_i = np.insert(self.r_i,0,0,axis=0)
                self.r_i = self.r_i[0:-1,:]

                self.r_i_sf = np.insert(self.r_i_sf,0,0,axis=0)
                self.r_i_sf = self.r_i_sf[0:-1,:]
                
                self.r_s_c = np.insert(self.r_s_c,0,0,axis=0)
                self.r_s_c = self.r_s_c[0:-1,:]
                
                self.r_s = np.insert(self.r_s,0,0,axis=0)
                self.r_s = self.r_s[0:-1]
                                
                
        return self.i, self.s_c, self.r_i, self.r_s_c
    
    

    